In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 1.8MB 5.2MB/s 
     |████████████████████████████████| 890kB 44.1MB/s 
     |████████████████████████████████| 3.2MB 40.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=57ac63fc3386919992eac1491c68a741b59785dd770d7f3ba01fe551ca049f2c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 5.4MB/s 


In [ ]:
import time
import torch
import random
import numpy as np
import pandas as pd
import transformers
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/Tesi/

/content/drive/MyDrive/Tesi


In [ ]:
from collections import Counter, defaultdict

def load_snli(fpaths):
    sa, sb, lb = [], [], []
    fpaths = np.atleast_1d(fpaths)
    for fpath in fpaths:
        with open(fpath) as fi:
            for line in fi:
                sample = json.loads(line)
                sa.append(sample['sentence1'])
                sb.append(sample['sentence2'])
                lb.append(sample['gold_label'])
    return sa, sb, lb

In [ ]:
import json

def prepare_snli(sa, sb, lb):
    
    classes = {"entailment", "contradiction"}
    anc_to_pairs = defaultdict(list)
    filtered = {}
    skipped = 0

    for xa, xb, y in zip(sa, sb, lb):
        anc_to_pairs[xa].append((xb, y))

    anchor_id = 0
    for anchor, payload in anc_to_pairs.items(): 
        
        filtered[anchor_id] = defaultdict(list)
        filtered[anchor_id]["anchor"].append(anchor)
        
        labels = set([t[1] for t in payload])
        
        if len(labels&classes) == len(classes):
            for text, label in payload:
                filtered[anchor_id][label].append(text)
            anchor_id += 1
        else:
            skipped += 1
            
    print("Loaded: {} \nSkipped: {}".format(anchor_id, skipped))
        
    return filtered

In [ ]:
train_data = ["dataset/snli_1.0_train.jsonl"]
test_data = ["dataset/snli_1.0_test.jsonl"]

tr_a, tr_b, tr_l = load_snli(train_data)
ts_a, ts_b, ts_l = load_snli(test_data)

fd_tr = prepare_snli(tr_a, tr_b, tr_l)
fd_ts = prepare_snli(ts_a, ts_b, ts_l)

Loaded: 149145 
Skipped: 1591
Loaded: 2921 
Skipped: 402


In [ ]:
train_df = pd.DataFrame(fd_tr)
train_df = train_df[:-1]
train_df = train_df.transpose()
train_df = train_df.dropna()
train_df = train_df[0:1000]
train_df

,anchor,neutral,contradiction,entailment
0,[A person on a horse jumps over a broken down ...,[A person is training his horse for a competit...,"[A person is at a diner, ordering an omelette.]","[A person is outdoors, on a horse.]"
1,[Children smiling and waving at camera],[They are smiling at their parents],[The kids are frowning],[There are children present]
2,[A boy is jumping on skateboard in the middle ...,[The boy is wearing safety equipment.],[The boy skates down the sidewalk.],[The boy does a skateboarding trick.]
3,[Two blond women are hugging one another.],[Some women are hugging on vacation.],[The women are sleeping.],[There are women showing affection.]
4,"[A few people in a restaurant setting, one of ...",[The people are eating omelettes.],[The people are sitting at desks in school.],[The diners are at a restaurant.]
...,...,...,...,...
1007,[A woman and kids swimming in a lake.],[The mother is having the kids race in the lake.],[A woman is making cookies for the kids.],[A woman in kids are outside playing in water.]
1008,[Some girls are dressed in zombie makeup.],[Girls dressed as zombie brides.],[These are zombies.],[Girls wearing zombie makeup.]
1009,[Four guys in striped shirts playing music ins...,[A group of Italians are playing instruments.],[A group of ladies is hunting an elephant.],[A band is playing songs indoors.]
1010,[A woman wearing black glasses is sharing swee...,"[The lady is babysitting her granddaughter., A...",[The woman and her grandson enjoy their day ou...,"[There is a young girl., The woman shared her ..."


In [ ]:
test_df = pd.DataFrame(fd_ts)
test_df = test_df[:-1]
test_df = test_df.transpose()
test_df = test_df.dropna()

In [ ]:
text = train_df.anchor.tolist()[0]
emb, mask = encode_text(text)

input_mask_expanded = mask.unsqueeze(-1).expand(emb.size()).float()
sum_embeddings = torch.sum(emb * input_mask_expanded, 1)
sum_mask = input_mask_expanded.sum(1)
output_vectors = sum_embeddings / sum_mask
fc = nn.Linear(512, 256)
out = fc(output_vectors)

NameError: ignored

In [ ]:
out.shape

torch.Size([1, 256])

In [ ]:
from transformers import T5Model, T5Tokenizer, T5EncoderModel, T5TokenizerFast, T5ForConditionalGeneration

tokenizer = T5TokenizerFast.from_pretrained('outputs')
model_encode = T5EncoderModel.from_pretrained('outputs', output_hidden_states = True)

def encode_text(text):
      print(text)
      if len(text) > 1:
        text = text[0]

      with torch.no_grad():    
          tokens = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, return_special_tokens_mask=True)
          input_ids = tokens.input_ids
          mask = 1 - tokens.special_tokens_mask 
          embedding = model_encode(input_ids=input_ids).last_hidden_state
          return embedding, mask

class SNLI(Dataset):
    def __init__(self, df, train=True):
        self.is_train = train
        self.anchors = df.iloc[:, 0].values
        
        if self.is_train:            
            self.neutral = df.iloc[:, 1].values
            self.negative = df.iloc[:, 2].values
            self.positive = df.iloc[:, 3].values
        else:
            self.images = df.values
        
    def __len__(self):
        return len(self.anchors)
    
    def __getitem__(self, item):

        if self.is_train:
            anchor = encode_text(self.anchors[item])
            positive = encode_text(self.positive[item])
            negative = encode_text(self.negative[item])
            neutral = encode_text(self.neutral[item])
            
            return anchor[0], anchor[1], positive[0], positive[1], negative[0], negative[1]
        
        else:
            anchor = encode_text(self.anchors[item])
            return anchor

Some weights of the model checkpoint at outputs were not used when initializing T5EncoderModel: ['decoder.embed_tokens.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0.layer.2.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.SelfAttention.v.weight', 'de

In [ ]:
embedding_dims = 2
batch_size = 32
epochs = 2

In [ ]:
train_ds = SNLI(train_df, train=True)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device.type == "cuda":
    torch.cuda.get_device_name()

In [ ]:
text = "hello world"
# Embedding
tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, return_special_tokens_mask=True)
input_ids = tokens.input_ids
mask = 1 - tokens.special_tokens_mask 
model = model_encode(input_ids=input_ids)
tokens_embedding = model.last_hidden_state

# Pooling
input_mask_expanded = mask.unsqueeze(-1).expand(tokens_embedding.size()).float()
sum_embeddings = torch.sum(tokens_embedding * input_mask_expanded, 1)
sum_mask = input_mask_expanded.sum(1)
output_vectors = sum_embeddings / sum_mask

# FC
m = nn.Linear(512, 256)
out = m(output_vectors)

In [ ]:
tokens_embedding.shape

torch.Size([1, 3, 512])

In [ ]:
mask.shape

torch.Size([1, 3])

In [ ]:
out.shape

torch.Size([1, 256])

In [ ]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_norm_difference(self, x1, x2):
        return torch.norm(x1 - x2)
    
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_norm_difference(anchor, positive)
        distance_negative = self.calc_norm_difference(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)
        return losses.mean()
        #return losses

In [ ]:
from transformers import T5Model, T5Tokenizer, T5EncoderModel, T5TokenizerFast, T5ForConditionalGeneration

class Network(nn.Module):

    def __init__(self):
        super(Network, self).__init__()
        
        self.fc = nn.Linear(512, 256)
        
    def forward(self, embedding, mask):
        '''
        x : documento passato alla rete
        
        # Embedding
        tokens = self.tokenizer(text, return_tensors="pt", padding=True, truncation=True, return_special_tokens_mask=True)
        input_ids = tokens.input_ids
        mask = 1 - tokens.special_tokens_mask 
        model = self.model_encode(input_ids=input_ids)
        tokens_embedding = model.last_hidden_state
        '''
        
        # Pooling
        input_mask_expanded = mask.unsqueeze(-1).expand(embedding.size()).float()
        sum_embeddings = torch.sum(embedding * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        output_vectors = sum_embeddings / sum_mask

        # FC
        
        out = self.fc(output_vectors)

        return out

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_normal_(m.weight)

In [ ]:
model = Network()
model.apply(init_weights)
model = torch.jit.script(model).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = torch.jit.script(TripletLoss())

In [ ]:
model.train()
for epoch in tqdm(range(epochs), desc="Epochs"):
    running_loss = []
    for step, (anchor, anchor_mask, positive, positive_mask, negative, negative_mask) in enumerate(tqdm(train_loader, desc="Training", leave=False)):
        anchor = anchor.to(device)
        positive = positive.to(device)
        negative = negative.to(device)

        anchor_mask = anchor_mask.to(device)
        positive_mask = positive_mask.to(device)
        negative_mask = negative_mask.to(device)
        
        optimizer.zero_grad()
        anchor_out = model(anchor, anchor_mask)
        positive_out = model(positive, positive_mask)
        negative_out = model(negative, negative_mask)
        
        loss = criterion(anchor_out, positive_out, negative_out)
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Loss: {:.4f}".format(epoch+1, epochs, np.mean(running_loss)))

['A stewardess reviews safety procedures on a plane.']
['Two large dogs greet other while their owners watch.']
['One biker is running with their bike while another is riding around them.']
['Young lady dressed in black shorts and light blue shirt sitting outside at a public table looking at a picture on her camera with her left hand on her face.']
['A young lady is looking at a picture.', 'A lady looks at a picture on her camera', 'Lady sits gazing at a camera.', 'A young lady is looking at her camera.', 'A lady at a table takes pictures.']
['People have bikes']
['A stewardess is on a plane.']
['the dogs see each other']
['A lady is using a cat to take pictures.', 'A young lady is riding her bike down a bike trail.', 'A lady looks at her phone.', 'Young lady sits in a private area.', 'A young lady is on her cellphone.']
['stray dogs are playing']
['The people are driving cars']
['The woman is on a boat.']
['A young lady is embarrassed at the pictures she took.', 'A young lady is looki

['Three young boys are dressed in hoodies facing the same way.']
['Several men sit outside on brick ledges built around tall trees.']
['The smiling ship captain is holding the wheel of his wooden ship.']
['a young girl in a flowery dress surrounded by watermelons']
['Some men are sitting outdoors.']
['There is a lot of fruit.']
['The ship captain is holding the sterring wheel.']
['Group of young boys all in hoodies.']
['The men are chopping down the trees.']
['There are only potatoes.']
['The ship captain has abandon his ship.']
['Three young boys are all in swimming trunks and no shirts.']
['The men just got done working.']
['The captain is on the bridge.']
['There are some citrus fruit.']
['Three best friends get into mischief.']
['Man singing on stage at a performance.']
['In the background, there are 3 people on the shore among several canoes, and the young man in the foreground is paddling through the water, while wearing a life jacket and smiling.']
['Kids building and destroying

In [ ]:
a, b = encode_text("today I live in Italy")

today I live in Italy


In [ ]:
texts = ["today I live in Italy", "tomorrow I will die"]

In [ ]:
tokens = tokenizer(texts, return_tensors="pt", padding='max_length', truncation=True, return_special_tokens_mask=True)

In [ ]:
out = []
for i in texts:
  with torch.no_grad():
    a, b = encode_text(i)
    out.append(model(a, b))

today I live in Italy
tomorrow I will die


In [ ]:
out

[tensor([[ 5.4619e-02,  4.3661e-02, -7.3686e-02, -5.3186e-03, -1.1612e-02,
           8.9405e-02,  1.3494e-01, -6.1315e-02,  7.0630e-02, -2.5729e-02,
           7.1664e-02,  6.5436e-02,  1.1219e-01,  7.9009e-02,  4.6449e-02,
          -7.1564e-02, -4.4925e-02, -9.3285e-03, -3.7592e-02,  7.4153e-02,
           5.9386e-02, -3.5775e-02, -2.6823e-02, -1.1766e-01, -5.5797e-03,
           1.0904e-01,  8.6748e-02,  2.0093e-01, -4.7282e-02, -3.4100e-02,
           4.2916e-02, -6.3839e-02,  5.3027e-02, -3.1599e-02, -1.2395e-01,
          -1.0804e-01,  1.8534e-03, -8.3625e-02, -5.1253e-02,  5.0831e-02,
           8.6875e-02,  6.0869e-02,  8.5395e-02,  1.8188e-02, -1.7866e-03,
          -6.2390e-02,  9.7675e-02, -6.8953e-03,  5.1820e-02, -8.6045e-03,
           3.5351e-02, -8.7311e-02,  1.7642e-02, -6.2992e-02,  4.6901e-03,
           5.8180e-04, -2.6896e-02, -4.0557e-03,  7.9466e-02, -6.5005e-02,
          -3.7689e-02,  1.0645e-01, -2.1521e-02,  6.4027e-02, -1.1740e-01,
           5.1976e-02, -2